In [1]:
# Standard library imports
from typing import Optional, Dict, Any, List, Tuple
import sys
import ast
from datetime import datetime

# Third-party imports
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import pandas as pd
import numpy as np

# Import your models (assuming they're in a models.py file)
from models import Recipe, RecipeAdd, User, Feedback, Review, UserReview

In [4]:
from elasticsearch import Elasticsearch

# Create Elasticsearch client
es = Elasticsearch(
    "http://localhost:9200",  # Changed from https to http
    basic_auth=("elastic", "pass"),  # Use your actual password
)
# Update disk watermark thresholds
es.cluster.put_settings(
    body={
        "persistent": {
            "cluster.routing.allocation.disk.watermark.low": "99%",
            "cluster.routing.allocation.disk.watermark.high": "99%",
            "cluster.routing.allocation.disk.watermark.flood_stage": "99%",
        }
    }
)
# Test connection
try:
    if es.ping():
        print("Successfully connected to Elasticsearch")
        print(es.info())
    else:
        print("Could not connect to Elasticsearch")
except Exception as e:
    print(f"Connection failed: {e}")

Successfully connected to Elasticsearch
{'name': 'e580306f495c', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'dpNy2-X0Qn2KylAhc84n0A', 'version': {'number': '8.17.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b6a7fed44faa321997703718f07ee0420804b41', 'build_date': '2024-12-11T12:08:05.663969764Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [21]:
# Elasticsearch mappings for the models

USER_MAPPING = {
    "mappings": {
        "properties": {
            "email": {"type": "keyword"},
            "name": {"type": "text"},
            "password": {"type": "keyword"},
            "embedding": {
                "type": "dense_vector",
                "dims": 768,  # Adjust dimension based on your embedding size
            },
        }
    }
}

RECIPE_MAPPING = {
    "mappings": {
        "properties": {
            "id": {"type": "integer"},
            "title": {"type": "text"},
            "ingredients": {"type": "text", "fields": {"keyword": {"type": "keyword"}}},
            "instructions": {"type": "text"},
            "prep_time": {"type": "integer"},
            "cook_time": {"type": "integer"},
            "cuisine": {"type": "keyword"},
            "course": {"type": "keyword"},
            "diet": {"type": "keyword"},
            "image": {"type": "keyword", "index": False},
            "url": {"type": "keyword", "index": False},
            "embedding": {
                "type": "dense_vector",
                "dims": 768,  # Adjust dimension based on your embedding size
                "similarity": "cosine",
            },
        }
    }
}

FEEDBACK_MAPPING = {
    "mappings": {
        "properties": {
            "email": {"type": "keyword"},
            "input_description": {"type": "text"},
            "input_image": {"type": "text", "index": False},
            "recipe_ids": {"type": "integer"},
            "rating": {"type": "integer"},
            "comment": {"type": "text"},
            "created_at": {"type": "date"},  # Added creation date
        }
    }
}

USER_REVIEW_MAPPING = {
    "mappings": {
        "properties": {
            "email": {"type": "keyword"},
            "reviews": {
                "type": "nested",
                "properties": {
                    "content": {
                        "type": "text"
                    },  # Changed from "text" to match the model
                    "created_at": {"type": "date"},  # Added creation date
                },
            },
        }
    }
}

RECIPE_ADD_MAPPING = {
    "mappings": {
        "properties": {
            "id": {"type": "integer"},
            "title": {"type": "text"},
            "ingredients": {"type": "text", "fields": {"keyword": {"type": "keyword"}}},
            "instructions": {"type": "text"},
            "prep_time": {"type": "integer"},
            "cook_time": {"type": "integer"},
            "cuisine": {"type": "keyword"},
            "course": {"type": "keyword"},
            "diet": {"type": "keyword"},
            "image": {"type": "keyword", "index": False},
            "url": {"type": "keyword", "index": False},
            "embedding": {
                "type": "dense_vector",
                "dims": 768,  # Adjust dimension based on your embedding size
            },
            "accepted": {"type": "boolean"},
        }
    }
}

In [23]:
# ... existing elasticsearch import and client setup ...
def create_indices(es_client):
    """Create all required indices if they don't exist"""
    for index_name, mapping in MAPPINGS.items():
        try:
            if not es_client.indices.exists(index=index_name):
                print(f"Creating index '{index_name}'...")
                es_client.indices.create(index=index_name, body=mapping)
                print(f"Successfully created index '{index_name}'")
            else:
                print(f"Index '{index_name}' already exists")
        except Exception as e:
            print(f"Error creating index '{index_name}': {e}")


# Define the mapping dictionary
MAPPINGS = {
    "users": USER_MAPPING,
    "recipes": RECIPE_MAPPING,
    "feedback": FEEDBACK_MAPPING,
    "user_reviews": USER_REVIEW_MAPPING,
    "recipe_additions": RECIPE_ADD_MAPPING,
}

# Create all indices
create_indices(es)

Creating index 'users'...
Successfully created index 'users'
Creating index 'recipes'...
Successfully created index 'recipes'
Creating index 'feedback'...
Successfully created index 'feedback'
Creating index 'user_reviews'...
Successfully created index 'user_reviews'
Creating index 'recipe_additions'...
Successfully created index 'recipe_additions'


In [22]:
def delete_indices(es_client):
    """Delete all indices defined in MAPPINGS"""
    for index_name in MAPPINGS.keys():
        try:
            if es_client.indices.exists(index=index_name):
                print(f"Deleting index '{index_name}'...")
                es_client.indices.delete(index=index_name)
                print(f"Successfully deleted index '{index_name}'")
            else:
                print(f"Index '{index_name}' does not exist")
        except Exception as e:
            print(f"Error deleting index '{index_name}': {e}")


# Delete all indices
# delete_indices(es)

Deleting index 'users'...
Successfully deleted index 'users'
Deleting index 'recipes'...
Successfully deleted index 'recipes'
Deleting index 'feedback'...
Successfully deleted index 'feedback'
Deleting index 'user_reviews'...
Successfully deleted index 'user_reviews'
Deleting index 'recipe_additions'...
Successfully deleted index 'recipe_additions'


In [5]:
def check_index_stats(es_client, index_name="recipes"):
    """
    Check if an index exists and get its document count

    Args:
        es_client: AsyncElasticsearch client
        index_name: Name of the index to check

    Returns:
        bool: True if index exists and has documents, False otherwise
    """
    try:
        # Check if index exists
        if not es_client.indices.exists(index=index_name):
            print(f"Index '{index_name}' does not exist!")
            return False

        # Get document count
        stats = es_client.count(index=index_name)
        doc_count = stats["count"]

        print(f"Index '{index_name}' contains {doc_count} documents")
        return doc_count > 0

    except Exception as e:
        print(f"Error checking index: {e}")
        return False


# Usage example:
has_documents = check_index_stats(es)
if not has_documents:
    print("Index is empty! You may need to index some documents first.")

Index 'recipes' contains 5044 documents


## read data and index it to elastic


In [24]:
import pandas as pd

df = pd.read_csv("./data.csv")
df.head()

,id,title,ingredients,instructions,prep_time,cook_time,cuisine,course,diet,image,url,embedding
0,4529,lavand-e-murgh recipe - afghani chicken in yog...,['fresh pomegranate fruit kernels few garnish'...,"['to begin making the lavand-e-murgh recipe, w...",15,25,Afghan,Dinner,High Protein Non Vegetarian,NaN,https://www.archanaskitchen.com/lavand-e-murgh...,"[[-0.0026710997335612774, 0.003612738568335771..."
1,4640,afghani dhoog recipe - cucumber mint buttermil...,"['cumin powder jeera', 'curd dahi yogurt', 'sa...",['to begin making the afghani dhoog recipe - c...,10,0,Afghan,Snack,Vegetarian,NaN,http://www.archanaskitchen.com/doogh-afghani-y...,"[[-0.014779524877667427, -0.008534302935004234..."
2,5978,malida recipe (healthy whole wheat afghan sweet),"['cardamom powder elaichi', 'dates pitted fine...","['to begin making the malida recipe, tear the ...",20,20,Afghan,Snack,Vegetarian,NaN,https://www.archanaskitchen.com/malida-recipe-...,"[[-0.01772255077958107, -0.019701037555933, -0..."
3,7092,moroccan spiced millet and lentil salad recipe,"['tomato chopped', 'extra virgin olive oil', '...",['to begin making the moroccan spiced millet a...,10,20,African,Dinner,Vegetarian,NaN,https://www.archanaskitchen.com/moroccan-spice...,"[[-0.06342744827270508, -0.01326711568981409, ..."
4,6684,chickpea & date tagine recipe,"['onion', 'cumin powder jeera', 'extra virgin ...",['to begin making the chickpea & date tagine r...,15,60,African,Dinner,High Protein Vegetarian,NaN,https://www.archanaskitchen.com/chickpea-date-...,"[[-0.03216571733355522, 0.029672250151634216, ..."


In [25]:
# Add import at the top
import ast

# Convert string representation to actual array when reading the embedding
df["embedding"] = df["embedding"].apply(ast.literal_eval)

# Now when you check the embedding, it will be an actual array
print(type(df["embedding"].loc[0]))  # Should print: <class 'list'>

<class 'list'>


index one document through recipe model


In [26]:
from elasticsearch import Elasticsearch
from typing import Dict, Any
import sys
from models import Recipe, Feedback

# ... existing code ...


def index_recipe_to_elastic(
    recipe: Recipe, es_client: Elasticsearch, index_name: str = "recipes"
) -> None:
    """
    Index a recipe to Elasticsearch

    Args:
        recipe: Recipe model instance
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index (default: "recipes")
    """
    doc = {
        "id": recipe.id,
        "title": recipe.title,
        "ingredients": recipe.ingredients,
        "instructions": recipe.instructions,
        "prep_time": recipe.prep_time,
        "cook_time": recipe.cook_time,
        "cuisine": recipe.cuisine,
        "course": recipe.course,
        "diet": recipe.diet,
        "image": str(recipe.image) if recipe.image else None,
        "url": str(recipe.url) if recipe.url else None,
        "embedding": recipe.embedding,
    }

    es_client.index(index=index_name, id=str(recipe.id), document=doc)
    print(f"Indexed recipe {recipe.id} to Elasticsearch")

bulk index df recipes


In [27]:
# Add necessary imports
from models import Recipe, RecipeAdd, User
from ast import literal_eval
import numpy as np
from elasticsearch.helpers import bulk

In [28]:
# Add necessary imports
from models import Recipe
from ast import literal_eval
import numpy as np


def row_to_recipe(row):
    """Convert a DataFrame row to a Recipe object"""
    # Convert embedding to a flat list of floats
    embedding = np.array(row.embedding).flatten().tolist()
    try:
        return Recipe(
            id=row.id,
            title=row.title,
            ingredients=(
                literal_eval(row.ingredients)
                if isinstance(row.ingredients, str)
                else row.ingredients
            ),
            instructions=(
                literal_eval(row.instructions)
                if isinstance(row.instructions, str)
                else row.instructions
            ),
            prep_time=row.prep_time,
            cook_time=row.cook_time,
            cuisine=row.cuisine,
            course=row.course,
            diet=row.diet,
            image=row.image if pd.notna(row.image) else None,
            url=row.url if pd.notna(row.url) else None,
            embedding=embedding,  # Now it's a flat list of floats
        )
    except Exception as e:
        return None

In [29]:
def bulk_index_recipe_batch(df_batch, es_client, index_name="recipes"):
    """
    Convert a batch of DataFrame rows to Recipe objects and bulk index them

    Args:
        df_batch: Pandas DataFrame batch containing recipes
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index
    """
    # Convert rows to Recipe objects and filter out None values
    recipes = [
        r
        for r in (row_to_recipe(row) for _, row in df_batch.iterrows())
        if r is not None
    ]

    if not recipes:
        print("No valid recipes in this batch")
        return
    # Prepare bulk indexing actions
    actions = []
    for recipe in recipes:
        # Build document with required fields
        doc = {
            "id": recipe.id,
            "title": recipe.title,
            "ingredients": recipe.ingredients,
            "instructions": recipe.instructions,
            "prep_time": recipe.prep_time,
            "cook_time": recipe.cook_time,
            "cuisine": recipe.cuisine,
            "course": recipe.course,
            "diet": recipe.diet,
        }

        # Only add optional fields if they're not None
        if recipe.image is not None:
            doc["image"] = recipe.image
        if recipe.url is not None:
            doc["url"] = recipe.url
        if recipe.embedding is not None:
            doc["embedding"] = recipe.embedding

        actions.append({"_index": index_name, "_id": str(recipe.id), "_source": doc})

    # Perform bulk indexing
    try:
        success, failed = bulk(es_client, actions, chunk_size=500, request_timeout=30)
        print(f"Successfully indexed {success} documents")
        if failed:
            print(f"Failed to index {len(failed)} documents")
    except Exception as e:
        print(f"Error during bulk indexing: {e}")


# Usage example - process in batches of 1000
batch_size = 1000
for start_idx in range(0, len(df), batch_size):
    batch = df.iloc[start_idx : start_idx + batch_size]
    bulk_index_recipe_batch(batch, es)

/tmp/ipykernel_5616/1292315588.py:48: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  success, failed = bulk(es_client, actions, chunk_size=500, request_timeout=30)


Successfully indexed 1000 documents
Successfully indexed 1000 documents
Successfully indexed 1000 documents
Successfully indexed 1000 documents
Successfully indexed 1000 documents
Successfully indexed 44 documents


In [111]:
def get_random_recipe(
    es_client: Elasticsearch, index_name: str = "recipes"
) -> tuple[Recipe, pd.DataFrame]:
    """
    Retrieve a random recipe from Elasticsearch, convert it to a Recipe model and DataFrame

    Args:
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index

    Returns:
        tuple: (Recipe model instance, pandas DataFrame)
    """
    # Random query
    random_query = {
        "query": {"function_score": {"query": {"match_all": {}}, "random_score": {}}},
        "size": 1,
    }

    try:
        # Execute search
        result = es_client.search(index=index_name, body=random_query)

        if not result["hits"]["hits"]:
            raise ValueError("No documents found in the index")

        # Convert to Recipe model
        hit = result["hits"]["hits"][0]["_source"]
        recipe = Recipe(
            id=hit["id"],
            title=hit["title"],
            ingredients=hit["ingredients"],
            instructions=hit["instructions"],
            prep_time=hit["prep_time"],
            cook_time=hit["cook_time"],
            cuisine=hit["cuisine"],
            course=hit["course"],
            diet=hit["diet"],
            image=hit.get("image"),
            url=hit.get("url"),
            embedding=hit.get("embedding"),
        )

        # Convert to DataFrame
        recipe_df = pd.DataFrame(
            [
                {
                    "id": recipe.id,
                    "title": recipe.title,
                    "ingredients": recipe.ingredients,
                    "instructions": recipe.instructions,
                    "prep_time": recipe.prep_time,
                    "cook_time": recipe.cook_time,
                    "cuisine": recipe.cuisine,
                    "course": recipe.course,
                    "diet": recipe.diet,
                    "embedding": recipe.embedding,
                    "url": recipe.url,
                }
            ]
        )

        return recipe, recipe_df

    except Exception as e:
        print(f"Error retrieving random recipe: {e}")
        return None, None


# Usage example:
recipe, df = get_random_recipe(es)
if recipe:
    print("Recipe Model:")
    display(recipe)
    print("\nDataFrame:")
    display(df)

Recipe Model:


Recipe(id=1001, title='Test Recipe', ingredients=['ingredient 1', 'ingredient 2'], instructions=['step 1', 'step 2'], prep_time=15, cook_time=30, cuisine='Italian', course='Main', diet='Vegetarian', image=None, url=None, embedding=None)


DataFrame:


,id,title,ingredients,instructions,prep_time,cook_time,cuisine,course,diet,embedding,url
0,1001,Test Recipe,"[ingredient 1, ingredient 2]","[step 1, step 2]",15,30,Italian,Main,Vegetarian,None,None


In [126]:
def get_random_pending_recipe(
    es_client: Elasticsearch, index_name: str = "recipe_additions"
) -> tuple[RecipeAdd, pd.DataFrame]:
    """
    Retrieve a random pending recipe from Elasticsearch

    Args:
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index

    Returns:
        tuple: (RecipeAdd model instance, pandas DataFrame)
    """
    # Random query
    random_query = {
        "query": {"function_score": {"query": {"match_all": {}}, "random_score": {}}},
        "size": 1,
    }

    try:
        # Execute search
        result = es_client.search(index=index_name, body=random_query)

        if not result["hits"]["hits"]:
            raise ValueError("No documents found in the index")

        # Convert to RecipeAdd model
        hit = result["hits"]["hits"][0]["_source"]
        recipe = RecipeAdd(
            id=hit["id"],
            title=hit["title"],
            ingredients=hit["ingredients"],
            instructions=hit["instructions"],
            prep_time=hit["prep_time"],
            cook_time=hit["cook_time"],
            cuisine=hit["cuisine"],
            course=hit["course"],
            diet=hit["diet"],
            image=hit.get("image"),
            url=hit.get("url"),
            embedding=hit.get("embedding"),
            accepted=hit.get("accepted", False),
        )

        # Convert to DataFrame
        recipe_df = pd.DataFrame(
            [
                {
                    "id": recipe.id,
                    "title": recipe.title,
                    "ingredients": recipe.ingredients,
                    "instructions": recipe.instructions,
                    "prep_time": recipe.prep_time,
                    "cook_time": recipe.cook_time,
                    "cuisine": recipe.cuisine,
                    "course": recipe.course,
                    "diet": recipe.diet,
                    "embedding": recipe.embedding,
                    "url": recipe.url,
                    "accepted": recipe.accepted,
                }
            ]
        )

        return recipe, recipe_df

    except Exception as e:
        print(f"Error retrieving random pending recipe: {e}")
        return None, None


# Usage example:
recipe, df = get_random_pending_recipe(es)
if recipe:
    print("Recipe Model:")
    display(recipe)
    print("\nDataFrame:")
    display(df)

Recipe Model:


RecipeAdd(id=2001, title='Homemade Pizza', ingredients=['2 cups all-purpose flour', '1 cup warm water', '2 tbsp olive oil', '1 tsp yeast', '1 tsp salt', 'Pizza toppings of choice'], instructions=['Mix flour, water, oil, yeast, and salt', 'Knead dough for 10 minutes', 'Let rise for 1 hour', 'Roll out and add toppings', 'Bake at 450°F for 15 minutes'], prep_time=70, cook_time=15, cuisine='Italian', course='Main Dish', diet='Vegetarian', image='https://example.com/pizza.jpg', url='https://example.com/homemade-pizza', embedding=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,


DataFrame:


,id,title,ingredients,instructions,prep_time,cook_time,cuisine,course,diet,embedding,url,accepted
0,2001,Homemade Pizza,"[2 cups all-purpose flour, 1 cup warm water, 2...","[Mix flour, water, oil, yeast, and salt, Knead...",70,15,Italian,Main Dish,Vegetarian,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",https://example.com/homemade-pizza,False


In [125]:
def index_pending_recipe(
    recipe: Recipe, es_client: Elasticsearch, index_name: str = "recipe_additions"
) -> None:
    """
    Convert Recipe to RecipeAdd and index it to Elasticsearch with accepted=False

    Args:
        recipe: Recipe model instance
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index for pending recipes
    """
    # Convert to pending RecipeAdd
    recipe_dict = recipe.model_dump()
    recipe_dict["accepted"] = False

    # Create new RecipeAdd instance
    pending_recipe = RecipeAdd(**recipe_dict)
    # Prepare document
    doc = pending_recipe.model_dump()

    try:
        # Index the document
        es_client.index(index=index_name, id=str(recipe.id), document=doc)
        print(f"Successfully indexed pending recipe {recipe.id} to {index_name}")
    except Exception as e:
        print(f"Error indexing pending recipe: {e}")


# Example usage:
# First, create a sample recipe
sample_recipe = Recipe(
    id=2001,
    title="Homemade Pizza",
    ingredients=[
        "2 cups all-purpose flour",
        "1 cup warm water",
        "2 tbsp olive oil",
        "1 tsp yeast",
        "1 tsp salt",
        "Pizza toppings of choice",
    ],
    instructions=[
        "Mix flour, water, oil, yeast, and salt",
        "Knead dough for 10 minutes",
        "Let rise for 1 hour",
        "Roll out and add toppings",
        "Bake at 450°F for 15 minutes",
    ],
    prep_time=70,
    cook_time=15,
    cuisine="Italian",
    course="Main Dish",
    diet="Vegetarian",
    image="https://example.com/pizza.jpg",
    url="https://example.com/homemade-pizza",
    embedding=[0.1] * 768,  # Dummy embedding
)

# Index the sample recipe
index_pending_recipe(sample_recipe, es)

# Verify it was indexed
result = es.get(index="recipe_additions", id=str(sample_recipe.id))
display(pd.DataFrame([result["_source"]]))

Successfully indexed pending recipe 2001 to recipe_additions


,id,title,ingredients,instructions,prep_time,cook_time,cuisine,course,diet,image,url,embedding,accepted
0,2001,Homemade Pizza,"[2 cups all-purpose flour, 1 cup warm water, 2...","[Mix flour, water, oil, yeast, and salt, Knead...",70,15,Italian,Main Dish,Vegetarian,https://example.com/pizza.jpg,https://example.com/homemade-pizza,"[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...",False


In [55]:
# Define the query
query = {
    "size": 0,
    "aggs": {"unique_categories": {"terms": {"field": "cuisine"}}},
}

# Execute the search
response = es.search(index="recipes", body=query)

# Print the aggregation results
print("Aggregation Results:")
print(response["aggregations"]["unique_categories"]["buckets"])

Aggregation Results:
[{'key': 'Indian', 'doc_count': 942}, {'key': 'Continental', 'doc_count': 805}, {'key': 'North Indian Recipes', 'doc_count': 571}, {'key': 'South Indian Recipes', 'doc_count': 414}, {'key': 'Italian Recipes', 'doc_count': 209}, {'key': 'Bengali Recipes', 'doc_count': 127}, {'key': 'Kerala Recipes', 'doc_count': 115}, {'key': 'Maharashtrian Recipes', 'doc_count': 108}, {'key': 'Fusion', 'doc_count': 106}, {'key': 'Karnataka', 'doc_count': 100}]


In [76]:
def initialize_globals():
    """Initialize global variables used across the application"""
    global df, distinct_ingredients, cuisines, courses, diets

    try:
        # Simple aggregation query for all fields
        query = {
            "size": 0,
            "aggs": {
                "unique_cuisines": {"terms": {"field": "cuisine", "size": 10000}},
                "unique_courses": {"terms": {"field": "course", "size": 10000}},
                "unique_diets": {"terms": {"field": "diet", "size": 10000}},
                "unique_ingredients": {
                    "terms": {"field": "ingredients.keyword", "size": 10000}
                },
            },
        }

        # Execute the search
        response = es.search(index="recipes", body=query)

        # Extract values from buckets
        cuisines = sorted(
            [
                bucket["key"]
                for bucket in response["aggregations"]["unique_cuisines"]["buckets"]
            ]
        )
        courses = sorted(
            [
                bucket["key"]
                for bucket in response["aggregations"]["unique_courses"]["buckets"]
            ]
        )
        diets = sorted(
            [
                bucket["key"]
                for bucket in response["aggregations"]["unique_diets"]["buckets"]
            ]
        )
        distinct_ingredients = sorted(
            [
                bucket["key"]
                for bucket in response["aggregations"]["unique_ingredients"]["buckets"]
            ]
        )

        print(
            f"Found {len(cuisines)} cuisines, {len(courses)} courses, {len(diets)} diets, "
            f"and {len(distinct_ingredients)} ingredients"
        )

        return distinct_ingredients, cuisines, courses, diets

    except Exception as e:
        print(f"Error initializing globals from Elasticsearch: {e}")
        return [], [], [], []

In [77]:
distinct_ingredients, cuisines, courses, diets = initialize_globals()
print("g")

Found 82 cuisines, 20 courses, 10 diets, and 10000 ingredients
g


user signup and user loging


In [19]:
def index_user(user: User, es_client: Elasticsearch, index_name: str = "users") -> bool:
    """
    Index a User model instance into Elasticsearch if it doesn't already exist

    Args:
        user: User model instance
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index for users (default: "users")

    Returns:
        bool: True if user was indexed successfully, False if user already exists or error occurs
    """
    try:
        # Check if user already exists
        if es_client.exists(index=index_name, id=user.email):
            print(f"User {user.email} already exists in {index_name}")
            return False

        # Convert User model to dictionary
        doc = user.model_dump()

        # Use email as document ID since it's unique
        es_client.index(index=index_name, id=user.email, document=doc)
        print(f"Successfully indexed user {user.email} to {index_name}")
        return True

    except Exception as e:
        print(f"Error indexing user: {e}")
        return False


# Example usage:
sample_user = User(
    email="test@exgample.com",
    name="Test User",
    password="hashed_password",  # In practice, this should be properly hashed
)

# Index the sample user
index_user(sample_user, es)

# Verify it was indexed (optional)
result = es.get(index="users", id=sample_user.email)
display(pd.DataFrame([result["_source"]]))

Successfully indexed user test@exgample.com to users


,email,name,password,embedding
0,test@exgample.com,Test User,hashed_password,None


In [5]:
def login_user(
    email: str, password: str, es_client: Elasticsearch, index_name: str = "users"
) -> bool:
    """
    Verify user credentials against Elasticsearch

    Args:
        email: User's email
        password: User's password (should be hashed in production)
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index for users (default: "users")

    Returns:
        bool: True if credentials are valid, False otherwise
    """
    try:
        # Check if user exists and get their data
        if not es_client.exists(index=index_name, id=email):
            print("User not found")
            return False

        # Get user data
        user_data = es_client.get(index=index_name, id=email)["_source"]

        # Check if password matches
        # NOTE: In production, you should use proper password hashing and verification
        if user_data["password"] == password:
            print("Login successful")
            return True
        else:
            print("Invalid password")
            return False

    except Exception as e:
        print(f"Error during login: {e}")
        return False


# Example usage:
success = login_user("test@exgample.com", "hashed_password", es)
print(f"Login successful: {success}")

Login successful
Login successful: True


In [6]:
# Test with correct credentials
success = login_user("test@example.com", "hashed_password", es)
print(f"Should succeed: {success}")

# Test with wrong password
success = login_user("test@example.com", "wrong_password", es)
print(f"Should fail: {success}")

# Test with non-existent user
success = login_user("nonexistent@example.com", "any_password", es)
print(f"Should fail: {success}")

User not found
Should succeed: False
User not found
Should fail: False
User not found
Should fail: False


feedback


In [7]:
def index_feedback(
    feedback: Feedback, es_client: Elasticsearch, index_name: str = "feedback"
) -> bool:
    """
    Index a Feedback model instance into Elasticsearch

    Args:
        feedback: Feedback model instance
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index (default: "feedback")

    Returns:
        bool: True if feedback was indexed successfully, False if error occurs
    """
    try:
        # Convert Feedback model to dictionary
        doc = feedback.model_dump()

        # Generate a unique ID (you might want to use a different strategy)
        doc_id = f"{feedback.email}_{feedback.created_at}"

        # Index the document
        es_client.index(index=index_name, id=doc_id, document=doc)
        print(
            f"Successfully indexed feedback from {feedback.email} at {feedback.created_at}"
        )
        return True

    except Exception as e:
        print(f"Error indexing feedback: {e}")
        return False


# Example usage:
sample_feedback = Feedback(
    email="test@example.com",
    input_description="I want a vegetarian pasta dish",
    recipe_ids=[1, 2, 3],
    rating=5,
    comment="Great recommendations!",
    created_at="2024-03-20T12:00:00",
)

# Index the sample feedback
success = index_feedback(sample_feedback, es)
print(f"Feedback indexing successful: {success}")

# Verify it was indexed (optional)
if success:
    result = es.get(
        index="feedback", id=f"{sample_feedback.email}_{sample_feedback.created_at}"
    )
    display(pd.DataFrame([result["_source"]]))

Successfully indexed feedback from test@example.com at 2024-03-20T12:00:00
Feedback indexing successful: True


,email,input_description,input_image,recipe_ids,rating,comment,created_at
0,test@example.com,I want a vegetarian pasta dish,None,"[1, 2, 3]",5,Great recommendations!,2024-03-20T12:00:00


user data and embeddings functions


In [15]:
# ... existing imports and models ...


def get_user_profile(
    email: str, es_client=es, index_name: str = "users"
) -> Optional[User]:
    """
    Retrieve a user profile from Elasticsearch by email and convert to User model

    Args:
        email: User's email address
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index (default: "users")

    Returns:
        Optional[User]: User model instance if found, None otherwise
    """
    try:
        # Get user data by email (used as document ID)
        result = es_client.get(index=index_name, id=email)

        # Convert Elasticsearch document to User model
        user_data = result["_source"]
        return User(**user_data)

    except Exception as e:
        print(f"Error retrieving user profile: {e}")
        return None


a = get_user_profile("test@exgample.com")

In [14]:
def update_user_embedding(
    email: str,
    embedding: List[float],
    es_client: Elasticsearch,
    index_name: str = "users",
) -> bool:
    """
    Update a user's embedding in Elasticsearch

    Args:
        email: User's email address (used as document ID)
        embedding: List of floats representing the user's new embedding
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index

    Returns:
        bool: True if update was successful, False otherwise
    """
    try:
        # Check if user exists
        if not es_client.exists(index=index_name, id=email):
            print(f"User {email} not found")
            return False

        # Update only the embedding field
        update_doc = {"doc": {"embedding": embedding}}

        es_client.update(index=index_name, id=email, body=update_doc)
        print(f"Successfully updated embedding for user {email}")
        return True

    except Exception as e:
        print(f"Error updating user embedding: {e}")
        return False


# Example usage:
new_embedding = [0.1] * 768  # Replace with actual embedding values
success = update_user_embedding("test@exgample.com", new_embedding, es)

Successfully updated embedding for user test@exgample.com


In [16]:
import torch

userembedding = torch.tensor(a.embedding, dtype=torch.float32)

In [5]:
def save_review(
    email: str,
    review: Review,  # Changed from review_text: str
    es_client: Elasticsearch = es,
    index_name: str = "user_reviews",
) -> bool:
    """
    Save a user review to Elasticsearch. If the user already has reviews, append to their list.
    If not, create a new document for the user.

    Args:
        email: User's email address
        review: Review object containing content and creation date
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index (default: "user_reviews")

    Returns:
        bool: True if review was saved successfully, False otherwise
    """
    try:
        # Check if user already has reviews
        if es_client.exists(index=index_name, id=email):
            # Get existing reviews
            result = es_client.get(index=index_name, id=email)
            user_reviews = result["_source"]["reviews"]

            # Append new review
            user_reviews.append(review.model_dump())

            # Update document
            update_doc = {"doc": {"reviews": user_reviews}}
            es_client.update(index=index_name, id=email, body=update_doc)
        else:
            # Create new UserReview document
            user_review = UserReview(email=email, reviews=[review])
            # Index new document
            es_client.index(
                index=index_name, id=email, document=user_review.model_dump()
            )

        print(f"Successfully saved review for user {email}")
        return True

    except Exception as e:
        print(f"Error saving review: {e}")
        return False

In [6]:
str(datetime.now())

'2024-12-26 09:16:18.184582'

In [7]:
datetime.now().__str__

<method-wrapper '__str__' of datetime.datetime object at 0x7f9924094de0>

In [8]:
from datetime import datetime
from models import Review

# Test saving a new review for a user
test_email = "test.user@example.com"
test_review = Review(content="This is my first review!", created_at=str(datetime.now()))

# First review - should create new document
success = save_review(test_email, test_review)
print(f"First review saved: {success}")

# Verify the review was saved
result = es.get(index="user_reviews", id=test_email)
print("\nAfter first review:")
display(pd.DataFrame([result["_source"]]))

# Add a second review - should update existing document
second_review = Review(
    content="This is my second review!", created_at=str(datetime.now())
)
success = save_review(test_email, second_review)
print(f"\nSecond review saved: {success}")

# Verify both reviews are present
result = es.get(index="user_reviews", id=test_email)
print("\nAfter second review:")
display(pd.DataFrame([result["_source"]]))

# Test with invalid email (optional error case)
try:
    invalid_email = "not-an-email"
    invalid_review = Review(content="This should fail", created_at=str(datetime.now()))
    save_review(invalid_email, invalid_review)
except Exception as e:
    print(f"\nExpected error with invalid email: {e}")

Successfully saved review for user test.user@example.com
First review saved: True

After first review:


,email,reviews
0,test.user@example.com,"[{'created_at': '2024-12-26T09:08:58.065329', ..."


Successfully saved review for user test.user@example.com

Second review saved: True

After second review:


,email,reviews
0,test.user@example.com,"[{'created_at': '2024-12-26T09:08:58.065329', ..."


Error saving review: 1 validation error for UserReview
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='not-an-email', input_type=str]


In [6]:
def get_user_reviews(
    email: str, es_client: Elasticsearch = es, index_name: str = "user_reviews"
) -> dict:
    """
    Get all reviews for a specific user from Elasticsearch

    Args:
        email: User's email address
        es_client: Elasticsearch client instance
        index_name: Name of the Elasticsearch index (default: "user_reviews")

    Returns:
        dict: Contains user's email and list of reviews, or empty reviews list if none found
    """
    try:
        # Check if user has any reviews
        if not es_client.exists(index=index_name, id=email):
            print(f"No reviews found for user {email}")
            return {"email": email, "reviews": []}

        # Get user's reviews
        result = es_client.get(index=index_name, id=email)
        return result["_source"]

    except Exception as e:
        print(f"Error retrieving reviews: {e}")
        return {"email": email, "reviews": []}


import pandas as pd

# Test the function
if __name__ == "__main__":
    # Test with a user that has reviews
    test_email = "review_test@example.com"
    reviews = get_user_reviews(test_email)
    print(f"\nReviews for {test_email}:")
    display(pd.DataFrame(reviews["reviews"]))

    # Test with a user that doesn't exist
    nonexistent_email = "nonexistent@example.com"
    reviews = get_user_reviews(nonexistent_email)
    print(f"\nReviews for {nonexistent_email}:")
    display(pd.DataFrame(reviews["reviews"]))


Reviews for review_test@example.com:


,created_at,content
0,2024-03-20T10:00:00,This is my first test review!
1,2024-03-20T11:00:00,This is my second test review!


No reviews found for user nonexistent@example.com

Reviews for nonexistent@example.com:


""


In [32]:
def filter_recipes_elastic(es_client=es, **kwargs) -> Tuple[List[Recipe], pd.DataFrame]:
    """
    Create and execute an Elasticsearch query based on filter parameters

    Args:
        es_client: Elasticsearch client instance
        **kwargs: Filter parameters (prep_time, cook_time, cuisine, course, diet, ingredients)

    Returns:
        Tuple[List[Recipe], pd.DataFrame]: Tuple containing:
            - List of matching Recipe model instances
            - DataFrame containing the same data
    """
    # Start with a match_all query
    query = {"bool": {"must": [], "filter": []}}

    for key, value in kwargs.items():
        if key == "Title" or not value:
            continue

        if key in ["prep_time", "cook_time"]:
            query["bool"]["filter"].append({"range": {key.lower(): {"lte": value}}})
        elif key in ["cuisine", "course", "diet"]:
            if isinstance(value, list) and value:
                query["bool"]["must"].append({"terms": {key.lower(): value}})
        elif key == "Cleaned_Ingredients" and value:
            query["bool"]["must"].append({"terms": {"ingredients.keyword": value}})

    # Construct the full search body
    search_body = {"query": query, "size": 100}

    try:
        # Execute the search
        response = es_client.search(index="recipes", body=search_body)

        # Convert hits to Recipe models and collect data for DataFrame
        recipes = []
        df_data = []

        for hit in response["hits"]["hits"]:
            source = hit["_source"]
            recipe = Recipe(
                id=source["id"],
                title=source["title"],
                ingredients=source["ingredients"],
                instructions=source["instructions"],
                prep_time=source["prep_time"],
                cook_time=source["cook_time"],
                cuisine=source["cuisine"],
                course=source["course"],
                diet=source["diet"],
                image=source.get("image"),
                url=source.get("url"),
                embedding=source.get("embedding"),
            )
            recipes.append(recipe)

            # Add the same data to the DataFrame collection
            df_data.append(
                {
                    "id": recipe.id,
                    "title": recipe.title,
                    "ingredients": recipe.ingredients,
                    "instructions": recipe.instructions,
                    "prep_time": recipe.prep_time,
                    "cook_time": recipe.cook_time,
                    "cuisine": recipe.cuisine,
                    "course": recipe.course,
                    "diet": recipe.diet,
                    "embedding": recipe.embedding,
                    "url": recipe.url,
                }
            )

        # Create DataFrame
        df = pd.DataFrame(df_data)

        return recipes, df

    except Exception as e:
        print(f"Error executing Elasticsearch query: {e}")
        return [], pd.DataFrame()


# Test the updated function
test_recipes, test_df = filter_recipes_elastic(
    # es,
    prep_time=20,
    cook_time=20,
    # cuisine=["Afghan"],
    course=["Dinner", "Snack", "Main Course"],
)
print(f"Found {len(test_recipes)} matching recipes")

if test_recipes:
    print("\nFirst matching recipe:")
    display(test_recipes[0])
    print("\nDataFrame head:")
    display(test_df.head())

Found 100 matching recipes

First matching recipe:


Recipe(id=4640, title='afghani dhoog recipe - cucumber mint buttermilk/ chaas', ingredients=['cumin powder jeera', 'curd dahi yogurt', 'salt taste', 'ice cubes few', 'cucumber peeled chopped', 'mint leaves pudina'], instructions=['to begin making the afghani dhoog recipe - cucumber mint buttermilk/ chaas into the preethi zodiac blender tall jar add yogurt, mint leaves, cucumber, cumin, ice cubes and salt to taste.turn on the puree function and blent to make a smooth drink.\xa0pour the\xa0afghani dhoog into serving glass and serve chilled.serve this\xa0afghani dhoog recipe - cucumber mint buttermilk/ chaas\xa0with\xa0lehsuni methi paneer\xa0and\xa0tawa paratha recipe - plain paratha\xa0for lunch.'], prep_time=10, cook_time=0, cuisine='Afghan', course='Snack', diet='Vegetarian', image=None, url='http://www.archanaskitchen.com/doogh-afghani-yogurt-drink-recipe-with-mint', embedding=[-0.014779524877667427, -0.008534302935004234, -0.0744883343577385, -0.03696717694401741, -0.030413435772061


DataFrame head:


,id,title,ingredients,instructions,prep_time,cook_time,cuisine,course,diet,embedding,url
0,4640,afghani dhoog recipe - cucumber mint buttermil...,"[cumin powder jeera, curd dahi yogurt, salt ta...",[to begin making the afghani dhoog recipe - cu...,10,0,Afghan,Snack,Vegetarian,"[-0.014779524877667427, -0.008534302935004234,...",http://www.archanaskitchen.com/doogh-afghani-y...
1,5978,malida recipe (healthy whole wheat afghan sweet),"[cardamom powder elaichi, dates pitted finely ...","[to begin making the malida recipe, tear the r...",20,20,Afghan,Snack,Vegetarian,"[-0.01772255077958107, -0.019701037555933, -0....",https://www.archanaskitchen.com/malida-recipe-...
2,7092,moroccan spiced millet and lentil salad recipe,"[tomato chopped, extra virgin olive oil, onion...",[to begin making the moroccan spiced millet an...,10,20,African,Dinner,Vegetarian,"[-0.06342744827270508, -0.01326711568981409, -...",https://www.archanaskitchen.com/moroccan-spice...
3,2096,marrakesh vegetable curry recipe,"[raisins few, badam almond few, onion chopped,...",[to begin making marrakesh vegetable curry rec...,15,20,African,Main Course,Vegetarian,"[-0.0607549324631691, 0.0676780492067337, -0.0...",http://www.archanaskitchen.com/marrakesh-veget...
4,3055,scrambled tofu noodles recipe,"[onion, mixed vegetables carrot, chilli vinega...",[to begin making the scrambled tofu noodles re...,10,10,Asian,Dinner,High Protein Vegetarian,"[-0.00255050347186625, -0.04626411944627762, -...",http://www.archanaskitchen.com/scrambled-tofu-...


In [36]:
ids = df["id"].tolist()

In [37]:
def get_embeddings(strings_list, api_url="http://localhost:8000/encode"):
    """
    Get embeddings for a list of strings (text or base64-encoded images)

    Args:
        strings_list (list): List of strings (text or base64-encoded images)
        api_url (str): URL of the embedding API

    Returns:
        list: List of embeddings
    """
    embeddings_list = []

    for string in strings_list:
        try:
            response = requests.post(api_url, json={"content": string})
            response.raise_for_status()
            result = response.json()

            if result["status"] == "success":
                embeddings_list.append(
                    {
                        "input": (
                            string[:50] + "..." if len(string) > 50 else string
                        ),  # Truncate long strings in log
                        "type": result["type"],
                        "embeddings": result["embeddings"],
                    }
                )
            else:
                print(
                    f"Error processing string: {result.get('message', 'Unknown error')}"
                )
                embeddings_list.append(
                    {
                        "input": string[:50] + "...",
                        "type": "error",
                        "error": result.get("message", "Unknown error"),
                    }
                )

        except Exception as e:
            print(f"Error in API call: {str(e)}")
            embeddings_list.append(
                {"input": string[:50] + "...", "type": "error", "error": str(e)}
            )

    return embeddings_list


def compute_average_embedding(title_text=None, image=None):
    embeddings = []
    if title_text:
        result = get_embeddings([title_text])[0]
        if result.get("type") != "error":
            title_embedding = np.array(result["embeddings"])
            embeddings.append(title_embedding)

    if image:
        # Convert PIL Image to base64 if needed
        if isinstance(image, Image.Image):
            buffered = BytesIO()
            image.save(buffered, format="PNG")
            img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
        else:
            # Assume it's already base64 encoded
            img_str = image

        result = get_embeddings([img_str])[0]
        if result.get("type") != "error":
            image_embedding = np.array(result["embeddings"])
            embeddings.append(image_embedding)

    if len(embeddings) == 0:
        raise ValueError("No valid embeddings could be generated from the input")

    # Ensure all embeddings have the same shape
    embeddings = [emb.flatten() for emb in embeddings]

    # Compute the average of all available embeddings
    avg_embedding = np.mean(embeddings, axis=0)

    return avg_embedding

In [53]:
import requests

embedding = compute_average_embedding("lentil soup", None)

In [ ]:
compute_average_embedding

In [54]:
def find_most_similar_recipe(
    avg_embedding, ids, top_n=5, min_similarity=0.7
) -> tuple[List[Recipe], pd.DataFrame]:
    """
    Find recipes from a specific set of IDs with embedding similarity > min_similarity

    Args:
        avg_embedding: Query embedding vector
        ids: List of recipe IDs to search within
        top_n: Maximum number of results to return (default 5)
        min_similarity: Minimum similarity threshold (default 0.7)

    Returns:
        tuple: (List[Recipe], pd.DataFrame) containing similar recipes
    """
    # Convert embedding to list if it's numpy array
    if hasattr(avg_embedding, "tolist"):
        avg_embedding = avg_embedding.tolist()

    # Query that combines ID filtering with similarity scoring
    query = {
        "query": {
            "script_score": {
                "query": {"terms": {"id": ids}},  # Filter by provided IDs
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {"query_vector": avg_embedding},
                },
            }
        },
        "min_score": 1 + 0.4,  # Add 1 to adjust for the +1.0 in script
        "size": top_n,
        "_source": True,  # Get all fields
    }

    try:
        # Execute search
        response = es.search(index="recipes", body=query)

        # Convert hits to Recipe models and collect data for DataFrame
        recipes = []
        df_data = []

        for hit in response["hits"]["hits"]:
            source = hit["_source"]
            similarity = hit["_score"] - 1.0  # Convert back to -1 to 1 range

            # Create Recipe model
            recipe = Recipe(
                id=source["id"],
                title=source["title"],
                ingredients=source["ingredients"],
                instructions=source["instructions"],
                prep_time=source["prep_time"],
                cook_time=source["cook_time"],
                cuisine=source["cuisine"],
                course=source["course"],
                diet=source["diet"],
                image=source.get("image"),
                url=source.get("url"),
                embedding=source.get("embedding"),
            )
            recipes.append(recipe)

            # Add data for DataFrame
            df_data.append(
                {
                    "id": recipe.id,
                    "title": recipe.title,
                    "ingredients": recipe.ingredients,
                    "instructions": recipe.instructions,
                    "prep_time": recipe.prep_time,
                    "cook_time": recipe.cook_time,
                    "cuisine": recipe.cuisine,
                    "course": recipe.course,
                    "diet": recipe.diet,
                    "embedding": recipe.embedding,
                    "url": recipe.url,
                    "similarity": similarity,
                }
            )

        # Create DataFrame
        df = pd.DataFrame(df_data)

        return df

    except Exception as e:
        print(f"Error finding similar recipes: {e}")
        return [], pd.DataFrame()

In [55]:
find_most_similar_recipe(embedding, ids)

,id,title,ingredients,instructions,prep_time,cook_time,cuisine,course,diet,embedding,url,similarity
0,3747,asian carrot ginger lentil soup recipe,"[carrot gajjar, onion, fresh red chilli, green...",[to begin making the asian carrot ginger lenti...,15,20,Asian,Dinner,High Protein Vegetarian,"[-0.07616453617811203, 0.03793567791581154, -0...",http://www.archanaskitchen.com/asian-carrot-gi...,0.675560
1,3284,chickpea soup recipe,"[onion chopped, of chickpea chickpea chickpeas...",[काबुली चना सूप रेसिपी बनाने के लिए सबसे पहले ...,15,20,North Indian Recipes,Dinner,High Protein Vegetarian,"[-0.08105280995368958, -0.018381595611572266, ...",https://www.archanaskitchen.com/chickpea-soup-...,0.653313
2,6825,moroccan lentil stew recipe with raisins,"[ginger grated, lukewarm water, sweet potato d...",[to start making moroccan lentil stew with rai...,15,30,African,Main Course,High Protein Vegetarian,"[-0.06751316785812378, 0.0513836070895195, -0....",https://www.archanaskitchen.com/moroccan-lenti...,0.627930
3,4780,carrot soup recipe,"[whole pepper crush make of vegetable stock, o...","[to make the carrot soup recipe, first heat th...",10,15,Continental,Dinner,Vegetarian,"[-0.07973533868789673, 0.03646014630794525, -0...",http://www.archanaskitchen.com/carrot-soup-rec...,0.617958
4,1438,lentil sauce recipe - stewed coconut milk sauce,"[fresh coconut grated, yellow moong dal split ...","[to begin making paruppu payasam, first heat g...",10,40,Kerala Recipes,Dessert,Vegetarian,"[-0.04027669504284859, -0.013138574548065662, ...",http://www.archanaskitchen.com/paruppu-payasam...,0.610544


asfasdasf
